In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from fastai.collab import *
from fastai.tabular.all import *

In [3]:
path = untar_data(URLs.ML_100k)

In [4]:
ratings = pd.read_csv(path/'u.data',delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
last_skywalker = np.array([0.98,0.9,-0.9])

In [6]:
user1 = np.array([0.9,0.8,-0.6])

In [10]:
c=(user1*last_skywalker)
c,c.sum()

(array([0.882, 0.72 , 0.54 ]), 2.1420000000000003)

In [11]:
casblanca = np.array([-0.99,-0.3,0.8])
c=(user1*casblanca)
c,c.sum()

(array([-0.891, -0.24 , -0.48 ]), -1.611)

In [12]:
movies = pd.read_csv(path/'u.item',delimiter='|',encoding='latin-1',
                    usecols=(0,1),names=('movie','title'),header=None)

In [13]:
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [14]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


In [15]:
dls = CollabDataLoaders.from_df(ratings,item_name = 'title', bs = 64)
dls.show_batch()

,user,title,rating
0,276,"Frighteners, The (1996)",4
1,1,Free Willy 2: The Adventure Home (1995),1
2,566,It's a Wonderful Life (1946),5
3,532,Rear Window (1954),5
4,487,Forrest Gump (1994),4
5,487,Young Guns II (1990),3
6,551,U Turn (1997),4
7,588,"Usual Suspects, The (1995)",5
8,234,"Usual Suspects, The (1995)",1
9,159,"Game, The (1997)",5


In [16]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5
user_factors = torch.randn(n_users,n_factors)
movie_factors = torch.randn(n_movies, n_factors)
user_factors,movie_factors

(tensor([[-0.4413, -0.8761, -2.1347,  0.9780,  0.8434],
         [-2.3448,  0.0400,  1.0758,  1.4110, -0.5440],
         [ 0.3586,  0.5881, -0.1314,  0.5481,  0.2836],
         ...,
         [-0.3741,  0.1074,  1.5706,  0.7585,  0.7347],
         [ 0.1813,  0.7106, -0.6455, -2.2118,  0.4095],
         [-1.9940,  0.0980, -1.0330,  1.4729,  0.2189]]),
 tensor([[ 0.3732, -1.2274,  0.1082, -0.9559, -1.0632],
         [ 1.0776,  0.5678, -0.3159,  0.2465,  1.1023],
         [ 0.1163,  2.0165,  0.9698, -0.2801,  0.3915],
         ...,
         [ 0.9125, -0.6627, -0.0940,  0.4249,  0.3388],
         [ 1.2032,  1.2949,  0.5266,  1.3638, -1.3315],
         [-1.2594,  0.5511, -0.9668, -2.1276,  1.0203]]))

In [17]:
user_factors.shape,movie_factors.shape

(torch.Size([944, 5]), torch.Size([1665, 5]))

In [18]:
one_hot_3 = one_hot(3,n_users).float()
user_factors.t() @ one_hot_3

tensor([-2.2673, -0.1524, -1.4737, -0.6770, -0.2380])

In [22]:
t=5
one_hot_t_test = one_hot(3,t).float()
one_hot_t_test

tensor([0., 0., 0., 1., 0.])

In [23]:
user_factors[3]

tensor([-2.2673, -0.1524, -1.4737, -0.6770, -0.2380])

In [24]:
class Example:
    def __init__(self,a):self.a=a
    def say(self,x):
        return f'Hello {self.a},{x}'

In [25]:
ex = Example('Sylvain')
ex.say("Nice to meet you")

'Hello Sylvain,Nice to meet you'

In [26]:
class DotProduct(Module):
    def __init__(self,n_users,n_movies,n_factors):
        self.user_factors = Embedding(n_users,n_factors)
        self.movie_factors = Embedding(n_movies,n_factors)

    def forward(self,x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return (users * movies).sum(dim=1)

In [27]:
x,y = dls.one_batch()
x.shape

torch.Size([64, 2])

In [28]:
model = DotProduct(n_users,n_movies,50)
learn = Learner(dls,model,loss_func=MSELossFlat())

In [29]:
learn.fit_one_cycle(5,5e-3)

epoch,train_loss,valid_loss,time
0,1.314448,1.326577,00:08
1,1.048542,1.114469,00:09
2,0.916021,0.992981,00:08
3,0.784251,0.904885,00:08
4,0.735292,0.879282,00:11


In [30]:
class DotProduct(Module):
    def __init__(self,n_users,n_movies,n_factors,y_range=(0,5.5)):
        self.user_factors = Embedding(n_users,n_factors)
        self.movie_factors = Embedding(n_movies,n_factors)
        self.y_range = y_range

    def forward(self,x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return sigmoid_range((users * movies).sum(dim=1),*self.y_range)

In [31]:
model = DotProduct(n_users,n_movies,50)
learn = Learner(dls,model,loss_func=MSELossFlat())
learn.fit_one_cycle(5,5e-3)

epoch,train_loss,valid_loss,time
0,0.932706,0.991100,00:10
1,0.672848,0.947579,00:09
2,0.471006,0.943816,00:10
3,0.351078,0.947436,00:09
4,0.348772,0.946301,00:11


In [32]:
class DotProduct(Module):
    def __init__(self,n_users,n_movies,n_factors,y_range=(0,5.5)):
        self.user_factors = Embedding(n_users,n_factors)
        self.user_bias = Embedding(n_users,1)
        self.movie_factors = Embedding(n_movies,n_factors)
        self.movie_bias = Embedding(n_movies,1)
        self.y_range = y_range

    def forward(self,x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        res = (users * movies).sum(dim=1,keepdim=True)
        res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
        return sigmoid_range(res,*self.y_range)

In [33]:
model = DotProduct(n_users,n_movies,50)
learn = Learner(dls,model,loss_func=MSELossFlat())
learn.fit_one_cycle(5,5e-3)

epoch,train_loss,valid_loss,time
0,0.859563,0.948985,00:12
1,0.586530,0.923571,00:10
2,0.397682,0.954888,00:11
3,0.326072,0.974242,00:09
4,0.296059,0.975108,00:11


In [35]:
# loss_with_wd = loss + wd *(parameters**2).sum()
# parameters.grad += wd*2*parameters

In [36]:
model = DotProduct(n_users,n_movies,50)
learn = Learner(dls,model,loss_func=MSELossFlat())
learn.fit_one_cycle(5,5e-3,wd=0.1)

epoch,train_loss,valid_loss,time
0,0.884248,0.942373,00:11
1,0.693656,0.893934,00:11
2,0.530747,0.874974,00:10
3,0.455214,0.864124,00:11
4,0.423633,0.858856,00:12


In [40]:
movie_bias = learn.model.movie_bias.weight.squeeze()
idxs = movie_bias.argsort()[:5]
[dls.classes['title'][i] for i in idxs]

['Lawnmower Man 2: Beyond Cyberspace (1996)',
 'Beautician and the Beast, The (1997)',
 'Crow: City of Angels, The (1996)',
 'Cable Guy, The (1996)',
 'Jury Duty (1995)']

In [41]:
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Good Will Hunting (1997)',
 'Titanic (1997)',
 'Star Wars (1977)',
 'Silence of the Lambs, The (1991)',
 'Shawshank Redemption, The (1994)']

In [42]:
learn = collab_learner(dls, n_factors=50,y_range=(0,5.5))
learn.fit_one_cycle(5,5e-3,wd=0.1)

epoch,train_loss,valid_loss,time
0,0.894752,0.954309,00:13
1,0.673758,0.896278,00:12
2,0.531767,0.884962,00:11
3,0.458322,0.871537,00:09
4,0.449064,0.865473,00:09


In [43]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1665, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1665, 1)
)

In [45]:
movie_factors = learn.model.i_weight.weight
idx = dls.classes['title'].o2i['Silence of the Lambs, The (1991)']
distances = nn.CosineSimilarity(dim=1)(movie_factors,movie_factors[idx][None])
idx = distances.argsort(descending=True)[1]
dls.classes['title'][idx]

'Fugitive, The (1993)'

In [46]:
embs = get_emb_sz(dls)
embs

[(944, 74), (1665, 102)]

In [50]:
class CollabNN1(Module):
    def __init__(self,user_sz,item_sz,y_range=(0,5.5),n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1],n_act),
            nn.ReLU(),
            nn.Linear(n_act,1)
        )
        self.y_range = y_range

    def forward(self,x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs,dim=1))
        return sigmoid_range(x,*self.y_range)

In [51]:
model = CollabNN1(*embs)

In [52]:
learn = Learner(dls,model,loss_func=MSELossFlat())
learn.fit_one_cycle(5,5e-3,wd=0.01)

epoch,train_loss,valid_loss,time
0,0.933553,0.971283,00:15
1,0.899119,0.921318,00:14
2,0.811038,0.911018,00:13
3,0.761834,0.879663,00:13
4,0.737531,0.871738,00:13


In [54]:
learn1 = collab_learner(dls,use_nn=True,y_range = (0,5.5),layers=[100,50])
learn1.fit_one_cycle(5,5e-3,wd=0.1)

epoch,train_loss,valid_loss,time
0,0.959857,0.985046,00:14
1,0.867088,0.945044,00:13
2,0.830372,0.903762,00:14
3,0.764927,0.889032,00:16
4,0.720946,0.881905,00:16


In [56]:
@delegates(TabularModel)
class EmbeddingNN(TabularModel):
    def __init__(self,emb_szs,layers,**kwargs):
        super().__init__(emb_szs,layers=layers,n_cont=0,out_sz=1,**kwargs)